### Imports

In [1]:
import pandas as pd

### Read data from 'xlsx' file

In [2]:
# Read 'xlsx' file as dataframe
df = pd.read_excel('../data/processed/data_cuba.xlsx', index_col=None, sheet_name='base_limpia')
# Create a list with dataframe column names
cols = df.columns

### Create pivot tables from dataframe

In [3]:
"""
Function to create 
pivot tables from
columns 90-99
"""
def create_pivot_table():
    dfs = []
    count = 90
    for i in range(10):
        # Create a pivot table from df
        table = pd.pivot_table(df, values=cols[0], index=cols[27],
                                columns=cols[count], aggfunc='count')
        count += 1
        # Convert index to columns and reset index
        table = table.rename_axis('Hospital').reset_index().rename_axis('', axis='columns')
        # Delete column and rename other column
        table = table.drop('No', 1).rename(columns={'Sí': f'Maltrato{i+1}'})

        # Remove rows with NaN values on column
        table.dropna(subset=[f'Maltrato{i+1}'], inplace=True)
        # Reset index
        table = table.reset_index(drop=True)
        # Append table to list
        dfs.append(table)
    return dfs

dfs = create_pivot_table()

m1, m2, m3, m4, m5, m6, m7, m8, m9, m10 = dfs[0], dfs[1], dfs[2], dfs[3], dfs[4], dfs[5], dfs[6], dfs[7], dfs[8], dfs[9]

/tmp/ipykernel_8093/2334071657.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  table = table.drop('No', 1).rename(columns={'Sí': f'Maltrato{i+1}'})


### Clean data

In [4]:
"""
Function to remove 
whitespace and new line
at the beginning
"""
def clean_dataframes(*args, **kwargs):
    dfs = list(args)
    for i in range(len(dfs)):
        dfs[i] = dfs[i]['Hospital'].replace(r'^\s|\n', '', regex=True, inplace=True)
    return dfs

clean_dataframes(m1, m2, m3, m4, m5, m6, m7, m8, m9, m10)

[None, None, None, None, None, None, None, None, None, None]

### Merge dataframes

In [5]:
"""
Function to merge dataframes
"""
def mergeDataframes(*args, **kwargs):
    dfs = list(args)
    for i in range(len(dfs)):
        if i <= 1:
            df = pd.merge(dfs[0], dfs[1], on='Hospital', how ='outer')
        else:
            df = pd.merge(df, dfs[i], on='Hospital', how ='outer')
    return df

maltratos = mergeDataframes(m1, m2, m3, m4, m5, m6, m7, m8, m9, m10)
maltratos['Hospital'].replace(r'^\s', '', regex=True, inplace=True)

maltratos.shape

(30, 11)

In [6]:
# Read 'xlsx' file as dataframe
directorio = pd.read_excel('../data/processed/directorio_hospitales.xlsx', index_col=None, sheet_name='base_limpia', converters={'Long':str, 'Lat': str})

# Merge dataframes by a column
df_join = pd.merge(maltratos, directorio, on='Hospital', how='left')

# Remove rows with NaN values on column
df_join.dropna(subset=['Long'], inplace=True)
# Reset index
df_join = df_join.reset_index(drop=True)

# Save dataframe as 'xlsx' file
# df_join.to_excel('../data/processed/maltratos_hospitales.xlsx', index = False)
# df_join.to_json('../data/processed/json/maltratos_hospitales.json', orient='index', force_ascii=False)

df_join

,Hospital,Maltrato1,Maltrato2,Maltrato3,Maltrato4,Maltrato5,Maltrato6,Maltrato7,Maltrato8,Maltrato9,Maltrato10,Long,Lat
0,Hospital General Docente 26 de Diciembre de Re...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,22.492379052036,-79.55278177
1,Hospital General Docente Dr. Agostinho Neto,1.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,20.1577360164937,-75.21762997
2,Hospital General Docente Vladimir Ilich Lenin,2.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,20.8882183791154,-76.26923029
3,Hospital Ginecobstétrico Materno Sur Mariana G...,3.0,1.0,5.0,6.0,5.0,NaN,1.0,NaN,NaN,NaN,21.617746757370714,-75.92184000604436
4,Hospital Ginecobstétrico Ramón González Coro,19.0,8.0,22.0,27.0,18.0,NaN,16.0,3.0,1.0,17.0,23.1309224772831,-82.39780127
5,Hospital Materno Infantil provincial Isabel Ma...,3.0,2.0,2.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,21.936654254801294,-79.4394423371609
6,Hospital Militar Luis Díaz Soto (Naval),3.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,NaN,23.153176268189828,-82.33258355
7,Hospital Universitario Abel Santamaría Cuadrado,3.0,2.0,3.0,4.0,4.0,1.0,3.0,NaN,1.0,1.0,22.44028611154951,-83.67122012
8,Hospital Clínico Quirúrgico Amalia Simoni,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,21.4043162072618,-77.8878039
9,Hospital Clínico Quirúrgico Juan Bruno Zayas,1.0,1.0,2.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,20.0346264574946,-75.79881625
